# Invariant Mass Prediction Using ML
## Bryan Dinh
### Comp_Phys Final Project 2023

Model Goal: Predict the invariant mass of a particle decay process which results in the creation of two muons. 

In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.config.list_physical_devices('GPU'))

[]


Load the training data

In [7]:
#os.chdir(os.path.dirname(BD_ML_Training.ipynb))
#os.chdir('../') # Get out of current folder

os.chdir(os.path.abspath("C:\\Users\\Bryan\\PycharmProjects\\comp_phys_2023\\Data")) # Dont hardcode
cwd=os.getcwd()
print(cwd)
train2=pd.read_csv(cwd+"\\trainDataset2.csv",index_col=0)
test=pd.read_csv(cwd+"\\FinalDataset.csv",index_col=0)
train2.head()

C:\Users\Bryan\PycharmProjects\comp_phys_2023\Data


,Pair Mass,Muon 1 Energy,Muon 2 Energy,Muon 1 pT,Muon 2 pT,Muon 1 Eta,Muon 2 Eta,Muon 1 Phi,Muon 2 Phi,Muon 1 Rapidity,Muon 2 Rapidity
25000,4.19125,5.06381,7.09718,4.57564,3.71216,0.457408,1.264380,3.039380,2.447480,0.457294,1.264040
25001,8.59466,5.27870,7.09718,4.45581,3.71216,0.598385,1.264380,-0.631795,2.447480,0.598235,1.264040
25002,22.63070,8.01042,27.69420,7.59714,5.36723,-0.328096,2.324520,-2.621600,-0.978774,-0.328065,2.324330
25003,12.34510,8.01042,5.55988,7.59714,5.32962,-0.328096,-0.292268,-2.621600,0.039000,-0.328065,-0.292212
25004,9.41510,8.01042,3.98217,7.59714,3.74132,-0.328096,-0.355897,-2.621600,1.475980,-0.328065,-0.355761


Normalizing the data

In [8]:
# Separate Target and Predictor Variables
TargetVariable=['Pair Mass']
Predictors=['Pair Mass','Muon 1 Energy','Muon 2 Energy',
            'Muon 1 pT','Muon 2 pT','Muon 1 Eta','Muon 2 Eta','Muon 1 Phi','Muon 2 Phi','Muon 1 Rapidity','Muon 2 Rapidity']

X=train2[Predictors].values
Y=train2[TargetVariable].values

X_test=test[Predictors].values
Y_test=test[TargetVariable].values
### Standardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(Y)


# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
Y=TargetVarScalerFit.transform(Y)
X=PredictorScalerFit.transform(X_test)
Y=TargetVarScalerFit.transform(Y_test)

In [9]:
from keras.models import Sequential
from keras.layers import Dense

# Create ANN Model
model=Sequential()

# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=11, kernel_initializer='normal', activation='relu'))

# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))

# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
          
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
model.fit(X, Y ,batch_size = 20, epochs = 50, verbose=1)

Epoch 1/50
6949/6949 [==============================] - 11s 1ms/step - loss: 0.1099
Epoch 2/50
6949/6949 [==============================] - 10s 1ms/step - loss: 0.0260
Epoch 3/50
6949/6949 [==============================] - 10s 1ms/step - loss: 0.0141
Epoch 4/50
6949/6949 [==============================] - 10s 1ms/step - loss: 0.0091
Epoch 5/50
6949/6949 [==============================] - 10s 1ms/step - loss: 0.0065
Epoch 6/50
6949/6949 [==============================] - 10s 1ms/step - loss: 0.0049
Epoch 7/50
6949/6949 [==============================] - 10s 1ms/step - loss: 0.0037
Epoch 8/50
6949/6949 [==============================] - 9s 1ms/step - loss: 0.0032
Epoch 9/50
6949/6949 [==============================] - 9s 1ms/step - loss: 0.0026
Epoch 10/50
6949/6949 [==============================] - 10s 1ms/step - loss: 0.0024
Epoch 11/50
6949/6949 [==============================] - 10s 1ms/step - loss: 0.0020
Epoch 12/50
6949/6949 [==============================] - 9s 1ms/step - loss:

In [14]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyperparameters to try
    batch_size_list = [15, 20]
    epoch_list = [20, 40]
    
    import pandas as pd
    SearchResultsDataList = []  # List to store data frames
    
    # Initializing the trials
    TrialNumber = 0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber += 1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

            # Defining the Second layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))

            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train, batch_size=batch_size_trial, epochs=epochs_trial, verbose=0)

            MAPE = np.mean(100 * (np.abs(y_test - model.predict(X_test)) / y_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:', 'batch_size:', batch_size_trial, '-', 'epochs:', epochs_trial, 'Accuracy:', 100-MAPE)
            
            # Append the current iteration's results to the list
            SearchResultsDataList.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]], columns=['TrialNumber', 'Parameters', 'Accuracy']))

    # Concatenate all the data frames in the list
    SearchResultsData = pd.concat(SearchResultsDataList, ignore_index=True)
    
    return SearchResultsData

# Calling the function
ResultsData = FunctionFindBestParams(X, Y, X_test, Y_test)


KeyboardInterrupt: 

Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

            # Defining the Second layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))

            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)

            MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)
            SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]], columns=['TrialNumber', 'Parameters', 'Accuracy']), ignore_index=True)

            #SearchResultsData=SearchResultsData(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]], columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X, Y, X_test, Y_test)